# Importar librerias

## Instalacion librerias

In [8]:
import pandas as pd
import re #Sirve para string

# Importación dataset

In [9]:
import pandas as pd
df = pd.read_csv("Crime_Data_from_2020_to_Present.csv")
df.head(10)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277
5,231808869,04/04/2023 12:00:00 AM,12/01/2020 12:00:00 AM,2300,18,Southeast,1826,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,9900 COMPTON AV,NaN,33.9467,-118.2463
6,230110144,04/04/2023 12:00:00 AM,07/03/2020 12:00:00 AM,900,1,Central,182,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1100 S GRAND AV,NaN,34.0415,-118.2620
7,220314085,07/22/2022 12:00:00 AM,05/12/2020 12:00:00 AM,1110,3,Southwest,303,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,2500 S SYCAMORE AV,NaN,34.0335,-118.3537
8,231309864,04/28/2023 12:00:00 AM,12/09/2020 12:00:00 AM,1400,13,Newton,1375,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1300 E 57TH ST,NaN,33.9911,-118.2521
9,211904005,12/31/2020 12:00:00 AM,12/31/2020 12:00:00 AM,1220,19,Mission,1974,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,9000 CEDROS AV,NaN,34.2336,-118.4535


In [ ]:
print(df.head(10))

In [ ]:
df.iloc[:,:14].head()


In [ ]:
df.iloc[:,14:].head()

In [ ]:
df.iloc[:,14:].tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Exploración de primeros 50 valores únicos solo para variables categóricas
for col in df.select_dtypes(include='object').columns:
    print(f"--- {col} ---")
    print(df[col].dropna().unique()[:50])  # Primeros 50 valores únicos
    print("\n")



In [10]:
# Vamos a renombrar las columnas para hacerlas más comprensibles, se trata de un cambio en el Diccionario
rename_columns = {
    'DR_NO': 'id_report',
    'Date Rptd': 'date_report',
    'DATE OCC': 'date_ocurrance',
    'TIME OCC': 'time_ocurrance',
    'AREA': 'area_code',
    'AREA NAME': 'area_name',
    'Rpt Dist No': 'district',
    'Part 1-2': 'crime_severity',
    'Crm Cd': 'crime_code',
    'Crm Cd Desc': 'crime_desc',
    'Mocodes': 'crime_mo',
    'Vict Age': 'age_victim',
    'Vict Sex': 'sex_victim',
    'Vict Descent': 'descent_victim',
    'Premis Cd': 'loc_code',
    'Premis Desc': 'loc_desc',
    'Weapon Used Cd': 'weapon_code',
    'Weapon Desc': 'weapon_desc',
    'Status': 'case_status_code',
    'Status Desc': 'case_status_desc',
    'Crm Cd 1': 'crime_code_1',
    'Crm Cd 2': 'crime_code_2',
    'Crm Cd 3': 'crime_code_3',
    'Crm Cd 4': 'crime_code_4',
    'LOCATION': 'location',
    'Cross Street': 'cross_street',
    'LAT': 'latitude',
    'LON': 'longitude'
}

# Renombrar las columnas
df.rename(columns=rename_columns, inplace=True)

# Confirmar el cambio
df.head()


,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,case_status_code,case_status_desc,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [ ]:
# Variables a explorar
columns_to_explore = [
    'area_code',
    'area_name',
    'district',
    'crime_severity',
    'crime_code',
    'crime_desc',
    'sex_victim',
    'descent_victim',
    'loc_code',
    'loc_desc',
    'weapon_code',
    'weapon_desc',
    'case_status_code',
    'case_status_desc'
]

# Mostrar TODOS los valores únicos para esas columnas
for col in columns_to_explore:
    if col in df.columns:
        print(f"--- {col} ---")
        print(sorted(df[col].dropna().unique()))
        print("\n")
    else:
        print(f"Column {col} not found in the dataframe.\n")


## **Vamos a trabajar en investigar la relación de códigos varios en el dataset. **

# 📄 Unique Values Summary

## area_code
`1, 2, 3, ..., 21`

## area_name
`'77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck', 'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic', 'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys', 'West LA', 'West Valley', 'Wilshire'`

## district
`101, 105, 109, 111, ..., 2199`  
(Muchos valores únicos de distrito, códigos de zonas específicas)

## crime_severity
`1, 2`

## crime_code
`110, 113, 121, 122, 210, 220, 230, 231, ..., 956`

## crime_desc
`'ARSON', 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER', 'BATTERY - SIMPLE ASSAULT', 'BIKE - STOLEN', 'BURGLARY', 'BURGLARY FROM VEHICLE', ..., 'WEAPONS POSSESSION/BOMBING'`
(Más de 100 tipos de crímenes diferentes)

## crime_mo
`Códigos tipo '0344 1822 1402', '0930', '0913 2024' etc.`  
(Modus Operandi combinados por código)

## sex_victim
`'-', 'F', 'H', 'M', 'X'`
(Indica sexo: Femenino, Masculino, Indeterminado, No reportado)

## descent_victim
`'-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z'`
(Origen étnico de la víctima)

## loc_code
`101.0, 102.0, 103.0, ..., 976.0`
(Muchos códigos de localización)

## loc_desc
`'STREET', 'BUS STOP', 'PARKING LOT', 'SINGLE FAMILY DWELLING', 'SCHOOL', 'HOSPITAL', 'HOTEL', 'MARKET', ..., 'YARD (RESIDENTIAL/BUSINESS)'`
(Lugares donde ocurrieron los crímenes)

## weapon_code
`101.0, 102.0, ..., 516.0`

## weapon_desc
`'HAND GUN', 'KNIFE WITH BLADE 6INCHES OR LESS', 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)', 'UNKNOWN FIREARM', 'VERBAL THREAT', ...`
(Descripción de armas usadas)

## case_status_code
`'AA', 'AO', 'CC', 'IC', 'JA', 'JO'`
(Códigos de estatus del caso)

## case_status_desc
`'Adult Arrest', 'Adult Other', 'Invest Cont', 'Juv Arrest', 'Juv Other', 'UNK'`
(Descripción del estatus del caso)


##1. Investigación de las áreas.

In [ ]:
# Primero aseguramos que district sea string para extraer area_code
df['district'] = df['district'].astype(str)

# Sacamos el primer dígito como área (puedes usar 1 o 2 dígitos según precisión)
df['area_code'] = df['district'].str[0].astype(int)

# Mapeo de area_code a area_name
area_mapping = {
    1: '77th Street',
    2: 'Central',
    3: 'Devonshire',
    4: 'Foothill',
    5: 'Harbor',
    6: 'Hollenbeck',
    7: 'Hollywood',
    8: 'Mission',
    9: 'N Hollywood',
    10: 'Newton',
    11: 'Northeast',
    12: 'Olympic',
    13: 'Pacific',
    14: 'Rampart',
    15: 'Southeast',
    16: 'Southwest',
    17: 'Topanga',
    18: 'Van Nuys',
    19: 'West LA',
    20: 'West Valley',
    21: 'Wilshire'
}

# Asignamos nombre al área
df['area_name'] = df['area_code'].map(area_mapping)

# Verificamos si el mapeo está correcto
print(df[['district', 'area_code', 'area_name']].head())

# Contar registros por area_name
print(df['area_name'].value_counts())


In [ ]:
import pandas as pd

# Definimos el listado completo de áreas (en el orden correcto)
area_names = [
    '77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck',
    'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic',
    'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys',
    'West LA', 'West Valley', 'Wilshire'
]

# Hacemos el conteo de registros por area_name
area_counts = df['area_name'].value_counts()

# Aseguramos que todos los nombres estén incluidos (incluso los que faltan)
area_counts = area_counts.reindex(area_names, fill_value=0)

# Ahora, buscamos las áreas con 0 crímenes
areas_sin_crimen = area_counts[area_counts == 0]

# Mostrar áreas sin crímenes
print("Áreas donde no hubo crímenes registrados:")
print(areas_sin_crimen)

In [ ]:
# Definimos las 21 áreas oficiales
area_names = [
    '77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck',
    'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic',
    'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys',
    'West LA', 'West Valley', 'Wilshire'
]

# 1. Conteo de crímenes por área
area_counts = df['area_name'].value_counts()

# 2. Asegurarnos que todas las áreas estén (rellenar con 0 si no existen)
area_counts = area_counts.reindex(area_names, fill_value=0)

# 3. Detectar áreas sin crímenes
areas_sin_crimen = area_counts[area_counts == 0].index.tolist()

print("Áreas donde no hubo crímenes:")
print(areas_sin_crimen)

# 4. Filtrar el df para filas con esas áreas (debería estar vacío, o muy pequeño si algo raro pasó)
df_sin_crimenes = df[df['area_name'].isin(areas_sin_crimen)]

# 5. Mostrar las filas
print(df_sin_crimenes)


Resumen:
9 áreas sí tienen crímenes (las que viste antes: 77th Street, Central, Devonshire, etc.).

12 áreas no tienen crímenes en tus datos actuales.

No hay errores de mapeo en district, area_code ni area_name.

## 2. Investigación Distritos.

In [ ]:
# Mostrar los valores únicos de la columna 'district'
if 'district' in df.columns:
    print("--- district ---")
    print(sorted(df['district'].dropna().unique()))
    print("\n")
else:
    print("Column 'district' not found in the dataframe.\n")

In [ ]:
# Asegurar que 'district' es string para extraer prefijos fácilmente
df['district'] = df['district'].astype(str)

# Extraer primer dígito (o primeros dos dígitos si quieres más precisión)
df['area_code'] = df['district'].str[0].astype(int)

# Opcionalmente, si quieres extraer dos primeros dígitos:
# df['area_code'] = df['district'].str[:2].astype(int)

# Definir los áreas válidas (1 al 21)
valid_areas = list(range(1, 22))

# Marcar si es un área válida
df['area_valid'] = df['area_code'].isin(valid_areas)

# Verificar resultados
print(df['area_valid'].value_counts())

# Ver ejemplos
print(df[['district', 'area_code', 'area_valid']].head())


In [ ]:
# Contar cuántos registros NO son válidos
invalid_count = (~df['area_valid']).sum()

print(f"Cantidad de registros con area_valid == False: {invalid_count}")


##3. Relación entre crime_code y crime_desc

Los crime_code y crime_desc que tienes provienen directamente de los reportes oficiales del LAPD (Los Angeles Police Department).

Cada "crime_code" es un tipo de crimen específico, basado en su severidad y en las leyes de California.

No es una clasificación inventada, es estándar dentro del departamento, usado para categorizar las denuncias.

In [ ]:
# Comprobar si un mismo crime_code tiene varias descripciones
crime_mapping = df[['crime_code', 'crime_desc']].drop_duplicates()

# Verificar duplicados de crime_code
duplicates = crime_mapping.duplicated(subset=['crime_code'], keep=False)
crime_mapping[duplicates]

Si no salen filas, la relación es uno a uno ✅.

In [ ]:
# Crear tabla resumen de códigos de crimen y su descripción
crime_table = df[['crime_code', 'crime_desc']].drop_duplicates().sort_values('crime_code')

# Mostrar la tabla
crime_table.reset_index(drop=True, inplace=True)
crime_table


In [ ]:
crime_table.to_csv('crime_code_description.csv', index=False)

Vamos a generar una agrupación de todos estos delitos en base al standard UCR (Uniform Crime Reporting)

In [ ]:
import pandas as pd

# Suponemos que tienes un DataFrame llamado 'df' con las columnas 'crime_code' y 'crime_desc'

# Mapeo de crime_codes a categorías UCR
ucr_mapping = {
    # Part I - Violent Crimes
    110: 'HOMICIDE',
    113: 'MANSLAUGHTER',
    121: 'RAPE',
    122: 'ATTEMPTED RAPE',
    815: 'SEXUAL PENETRATION',
    820: 'ORAL COPULATION',
    821: 'SODOMY',
    210: 'ROBBERY',
    220: 'ROBBERY ATTEMPTED',
    230: 'AGGRAVATED ASSAULT',
    231: 'AGGRAVATED ASSAULT',
    235: 'AGGRAVATED ASSAULT',
    236: 'AGGRAVATED ASSAULT',
    250: 'AGGRAVATED ASSAULT',
    251: 'AGGRAVATED ASSAULT',
    761: 'AGGRAVATED ASSAULT',
    926: 'AGGRAVATED ASSAULT',
    435: 'SIMPLE ASSAULT',
    436: 'SIMPLE ASSAULT',
    437: 'SIMPLE ASSAULT',
    622: 'SIMPLE ASSAULT',
    623: 'SIMPLE ASSAULT',
    624: 'SIMPLE ASSAULT',
    625: 'SIMPLE ASSAULT',
    626: 'SIMPLE ASSAULT',
    627: 'SIMPLE ASSAULT',
    647: 'SIMPLE ASSAULT',
    763: 'SIMPLE ASSAULT',
    928: 'SIMPLE ASSAULT',
    930: 'SIMPLE ASSAULT',

    # Part I - Property Crimes
    310: 'BURGLARY',
    320: 'BURGLARY',
    510: 'MOTOR VEHICLE THEFT',
    520: 'MOTOR VEHICLE THEFT',
    433: 'MOTOR VEHICLE THEFT',
    330: 'THEFT FROM VEHICLE',
    331: 'THEFT FROM VEHICLE',
    410: 'THEFT FROM VEHICLE',
    420: 'THEFT FROM VEHICLE',
    421: 'THEFT FROM VEHICLE',
    350: 'PERSONAL THEFT',
    351: 'PERSONAL THEFT',
    352: 'PERSONAL THEFT',
    353: 'PERSONAL THEFT',
    450: 'PERSONAL THEFT',
    451: 'PERSONAL THEFT',
    452: 'PERSONAL THEFT',
    453: 'PERSONAL THEFT',
    341: 'OTHER THEFT',
    343: 'OTHER THEFT',
    345: 'OTHER THEFT',
    440: 'OTHER THEFT',
    441: 'OTHER THEFT',
    442: 'OTHER THEFT',
    443: 'OTHER THEFT',
    444: 'OTHER THEFT',
    445: 'OTHER THEFT',
    470: 'OTHER THEFT',
    471: 'OTHER THEFT',
    472: 'OTHER THEFT',
    473: 'OTHER THEFT',
    474: 'OTHER THEFT',
    475: 'OTHER THEFT',
    480: 'OTHER THEFT',
    485: 'OTHER THEFT',
    487: 'OTHER THEFT',
    491: 'OTHER THEFT',

    # Part II - Other Crimes
    500: 'DRUG OFFENSES',
    501: 'DRUG OFFENSES',
    502: 'DRUG OFFENSES',
    503: 'DRUG OFFENSES',
    504: 'DRUG OFFENSES',
    505: 'DRUG OFFENSES',
    506: 'DRUG OFFENSES',
    600: 'SIMPLE ASSAULT',
    601: 'SIMPLE ASSAULT',
    602: 'SIMPLE ASSAULT',
    603: 'SIMPLE ASSAULT',
    700: 'VANDALISM',
    701: 'VANDALISM',
    702: 'WEAPONS OFFENSE',
    703: 'WEAPONS OFFENSE',
    704: 'WEAPONS OFFENSE',
    705: 'WEAPONS OFFENSE',
    800: 'SEX OFFENSES (NON-RAPE)',
    801: 'SEX OFFENSES (NON-RAPE)',
    802: 'SEX OFFENSES (NON-RAPE)',
    803: 'SEX OFFENSES (NON-RAPE)',
    900: 'OTHER OFFENSES',
    901: 'OTHER OFFENSES',
    902: 'OTHER OFFENSES',
    903: 'OTHER OFFENSES',
    904: 'OTHER OFFENSES',
    905: 'OTHER OFFENSES',
    906: 'OTHER OFFENSES',
    907: 'OTHER OFFENSES'
}

# Crear nueva columna 'crime_ucr' basada en el mapping
df['crime_ucr'] = df['crime_code'].map(ucr_mapping).fillna('UNKNOWN')

# Si quieres ver los resultados
print(df[['crime_code', 'crime_desc', 'crime_ucr']].head())


In [ ]:
df.columns

## 4. Investigación de Sexo.

In [ ]:
# 1. Primero, corregimos H -> M
df['sex_victim'] = df['sex_victim'].replace('H', 'M')

# 2. Definimos la lista válida final
valores_validos_sexo = ['-', 'F', 'M', 'X']

# 3. Comprobamos de nuevo si queda todo limpio
print("Valores únicos en sex_victim después de la corrección:")
print(df['sex_victim'].unique())

# 4. Opcional: identificar si queda algo inválido
valores_invalidos = df[~df['sex_victim'].isin(valores_validos_sexo)]

print(f"\nCantidad de registros con sex_victim inválido después de corregir: {len(valores_invalidos)}")


In [ ]:
# Reemplazar nulos por '-'
df['sex_victim'] = df['sex_victim'].fillna('-')

# Volver a verificar
print("Valores únicos en sex_victim después de rellenar NaN:")
print(df['sex_victim'].unique())

In [ ]:
# 1. Conteo de sex_victim
conteo_sexo = df['sex_victim'].value_counts()

# 2. Porcentaje de sex_victim
porcentaje_sexo = df['sex_victim'].value_counts(normalize=True) * 100

# 3. Mostrar resultados
resultado = pd.DataFrame({
    'Cantidad': conteo_sexo,
    'Porcentaje (%)': porcentaje_sexo.round(2)
})

print(resultado)

## 5. Origen

In [ ]:
# 1. Definimos los valores válidos
valores_validos_descent = ['-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 
                           'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z']

# 2. Verificamos los valores únicos en el dataset
print("Valores únicos en descent_victim:")
print(df['descent_victim'].unique())

# 3. Detectamos valores inválidos
valores_invalidos = df[~df['descent_victim'].isin(valores_validos_descent)]

# 4. Mostramos las filas con descent_victim inválido
print("\nFilas con valores inválidos en descent_victim:")
print(valores_invalidos[['id_report', 'descent_victim']])

# 5. Cantidad de registros inválidos
print(f"\nCantidad de registros con descent_victim inválido: {len(valores_invalidos)}")

In [ ]:
# Reemplazar NaN por '-'
df['descent_victim'] = df['descent_victim'].fillna('-')

In [ ]:
# 1. Definir los valores válidos de razas
valores_validos_descent = ['-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 
                           'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z']

# 2. Rellenar nulos si es necesario
df['descent_victim'] = df['descent_victim'].fillna('-')

# 3. Conteo de víctimas por raza
conteo_raza = df['descent_victim'].value_counts()

# 4. Porcentaje por raza
porcentaje_raza = df['descent_victim'].value_counts(normalize=True) * 100

# 5. Crear el DataFrame resultado
resultado_raza = pd.DataFrame({
    'Cantidad': conteo_raza,
    'Porcentaje (%)': porcentaje_raza.round(2)
})

# 6. Mostrar resultados
print(resultado_raza)

## Investigación - Case Status Code/Description
## 6. Localización del suceso

In [139]:
mapping_loc_code_desc = {
    101.0: 'STREET',
    102.0: 'BUS STOP',
    103.0: 'PARKING LOT',
    104.0: 'DRIVEWAY',
    105.0: 'SIDEWALK',
    201.0: 'SINGLE FAMILY DWELLING',
    202.0: 'MULTI-FAMILY DWELLING (APARTMENT)',
    203.0: 'DUPLEX/TRIPLEX',
    204.0: 'ROOMING HOUSE',
    301.0: 'GOVERNMENT BUILDING',
    302.0: 'POLICE FACILITY',
    303.0: 'JAIL/PRISON',
    401.0: 'RESTAURANT',
    402.0: 'BAR/NIGHTCLUB',
    501.0: 'HOSPITAL/MEDICAL FACILITY',
    502.0: 'NURSING HOME',
    601.0: 'HOTEL/MOTEL',
    602.0: 'MOTOR HOTEL',
    701.0: 'SCHOOL, PUBLIC',
    702.0: 'SCHOOL, PRIVATE',
    801.0: 'BANK',
    802.0: 'ATM',
    901.0: 'MARKET',
    902.0: 'CONVENIENCE STORE',
    903.0: 'DEPARTMENT STORE',
    904.0: 'SHOPPING MALL',
    905.0: 'GAS STATION',
    906.0: 'AUTO DEALERSHIP',
    907.0: 'PARK',
    908.0: 'PLAYGROUND',
    909.0: 'GYM/FITNESS CENTER',
    910.0: 'THEATER',
    911.0: 'MUSEUM',
    912.0: 'LIBRARY',
    913.0: 'CHURCH/SYNAGOGUE/TEMPLE',
    914.0: 'WAREHOUSE',
    915.0: 'FACTORY/PLANT',
    916.0: 'CONSTRUCTION SITE',
    917.0: 'VACANT LOT',
    918.0: 'ABANDONED BUILDING',
    919.0: 'BUSINESS OFFICE',
    920.0: 'YARD (RESIDENTIAL/BUSINESS)',
    976.0: 'OTHER LOCATION'
}


In [ ]:
# Mapear el loc_desc estándar según loc_code
df['loc_desc'] = df['loc_code'].map(mapping_loc_code_desc).fillna('UNKNOWN')

In [ ]:
# 1. Contar crímenes por localización (loc_desc)
conteo_localizaciones = df['loc_desc'].value_counts()

# 2. Sacar porcentaje
porcentaje_localizaciones = (conteo_localizaciones / conteo_localizaciones.sum()) * 100

# 3. Juntar en un solo DataFrame
resultado_localizaciones = pd.DataFrame({
    'Cantidad': conteo_localizaciones,
    'Porcentaje (%)': porcentaje_localizaciones.round(2)
})

# 4. Tomar solo el TOP 10
top_10_localizaciones = resultado_localizaciones.head(10)

# 5. Mostrar
print(top_10_localizaciones)



# 7. Investigación sobre las armas

In [6]:
mapping_weapon_code_desc = {
    100.0: 'HAND GUN',
    101.0: 'RIFLE',
    102.0: 'SHOTGUN',
    103.0: 'ASSAULT WEAPON (AK47, UZI, etc.)',
    104.0: 'OTHER FIREARM',
    105.0: 'SEMI-AUTOMATIC HANDGUN',
    106.0: 'SEMI-AUTOMATIC RIFLE',
    200.0: 'KNIFE/SHARP INSTRUMENT',
    201.0: 'RAZOR',
    202.0: 'SCISSORS',
    203.0: 'ICE PICK',
    204.0: 'SWITCH BLADE',
    205.0: 'OTHER CUTTING INSTRUMENT',
    206.0: 'SWORD/MACHETE',
    300.0: 'BLUNT OBJECT (CLUB, BAT, etc.)',
    301.0: 'ROCK/BRICK',
    302.0: 'PIPE/METAL BAR',
    303.0: 'STICK/POLE',
    400.0: 'PERSONAL WEAPONS (HANDS, FISTS, FEET, ETC.)',
    500.0: 'OTHER WEAPON',
    501.0: 'BOTTLE',
    502.0: 'CHAIN',
    503.0: 'ROPE/STRING/SHOELACE',
    504.0: 'CHEMICAL SPRAY (MACE/PEPPER SPRAY)',
    505.0: 'FIRE (ARSON)',
    506.0: 'EXPLOSIVES',
    507.0: 'VEHICLE',
    508.0: 'AXE',
    509.0: 'SCREWDRIVER',
    510.0: 'UNKNOWN WEAPON',
    700.0: 'SIMULATED WEAPON (FAKE GUN, ETC.)',
    800.0: 'VERBAL THREAT',
    900.0: 'OTHER DANGEROUS WEAPON',
}

In [7]:
# 1. Aseguramos que 'weapon_code' esté correctamente mapeado
df['weapon_desc'] = df['weapon_code'].map(mapping_weapon_code_desc).fillna('UNKNOWN')

# 2. Conteo de armas por descripción
conteo_armas = df['weapon_desc'].value_counts()

# 3. Porcentaje de armas
porcentaje_armas = (conteo_armas / conteo_armas.sum()) * 100

# 4. Crear el DataFrame final
resultado_armas = pd.DataFrame({
    'Cantidad': conteo_armas,
    'Porcentaje (%)': porcentaje_armas.round(2)
})

# 5. Tomar el TOP 10
top_10_armas = resultado_armas.head(10)

# 6. Mostrar
print(top_10_armas)



NameError: name 'df' is not defined


8. Investigación sobre Modus Operandi


Valores numericos (Aaron y Javi)


### id_report

In [11]:
df['id_report'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 986500 entries, 0 to 986499
Series name: id_report
Non-Null Count   Dtype
--------------   -----
986500 non-null  int64
dtypes: int64(1)
memory usage: 7.5 MB


In [12]:
df.iloc[:, :5].head(10)

,id_report,date_report,date_ocurrance,time_ocurrance,area_code
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6
5,231808869,04/04/2023 12:00:00 AM,12/01/2020 12:00:00 AM,2300,18
6,230110144,04/04/2023 12:00:00 AM,07/03/2020 12:00:00 AM,900,1
7,220314085,07/22/2022 12:00:00 AM,05/12/2020 12:00:00 AM,1110,3
8,231309864,04/28/2023 12:00:00 AM,12/09/2020 12:00:00 AM,1400,13
9,211904005,12/31/2020 12:00:00 AM,12/31/2020 12:00:00 AM,1220,19


In [15]:
df.loc[df['id_report']<9999999]

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,case_status_code,case_status_desc,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
39890,817,09/20/2020 12:00:00 AM,09/19/2020 12:00:00 AM,1700,17,Devonshire,1777,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,9100 RUBIO AV,NaN,34.2367,-118.4955
268033,2113,06/21/2021 12:00:00 AM,04/05/2021 12:00:00 AM,1835,13,Newton,1385,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,800 E GAGE AV,NaN,33.9820,-118.2622
581052,2203,12/31/2022 12:00:00 AM,12/19/2022 12:00:00 AM,1930,3,Southwest,356,1,522,"VEHICLE, STOLEN - OTHER (MOTORIZED SCOOTERS, B...",...,IC,Invest Cont,522.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
734486,2315,07/23/2023 12:00:00 AM,07/22/2023 12:00:00 AM,1800,15,N Hollywood,1566,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,IC,Invest Cont,420.0,NaN,NaN,NaN,4400 ENSIGN AV,NaN,34.1504,-118.3712
876571,2401,01/10/2024 12:00:00 AM,12/21/2023 12:00:00 AM,1930,14,Pacific,1416,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,3400 CABRILLO BL,NaN,34.0099,-118.4453


### date_report

In [16]:
df['date_report']=pd.to_datetime(df['date_report'])

C:\Users\aaron\AppData\Local\Temp\ipykernel_27988\240740369.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_report']=pd.to_datetime(df['date_report'])


In [17]:
df['date_report'].head()

0   2020-03-01
1   2020-02-09
2   2020-11-11
3   2023-05-10
4   2022-08-18
Name: date_report, dtype: datetime64[ns]

In [18]:
df['date_report'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 986500 entries, 0 to 986499
Series name: date_report
Non-Null Count   Dtype         
--------------   -----         
986500 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.5 MB


### date_ocurrance

In [19]:
df['date_ocurrance']=pd.to_datetime(df['date_ocurrance'])

C:\Users\aaron\AppData\Local\Temp\ipykernel_27988\779857604.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_ocurrance']=pd.to_datetime(df['date_ocurrance'])


In [20]:
df['date_report'].head()

0   2020-03-01
1   2020-02-09
2   2020-11-11
3   2023-05-10
4   2022-08-18
Name: date_report, dtype: datetime64[ns]

In [21]:
df['date_report'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 986500 entries, 0 to 986499
Series name: date_report
Non-Null Count   Dtype         
--------------   -----         
986500 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.5 MB


### time_ocurrance

In [22]:
df['time_ocurrance'] = df['time_ocurrance'].astype(str).str.zfill(4) #Asi pongo 4 cifras en la columna

In [23]:
df['time_ocurrance'] = df['time_ocurrance'].str[:-2] + ":" + df['time_ocurrance'].str[-2:] # Aqui converti a string y luego lo separe lo 2 primero numeros, luego : y luego los 2 ultimos

In [25]:
df['time_ocurrance'] = pd.to_datetime(df['time_ocurrance'], format ='%H:%M').dt.time

In [26]:
df.head(10)

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,case_status_code,case_status_desc,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
0,190326475,2020-03-01,2020-03-01,21:30:00,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,2020-02-09,2020-02-08,18:00:00,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,2020-11-11,2020-11-04,17:00:00,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,2023-05-10,2020-03-10,20:37:00,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,2022-08-18,2020-08-17,12:00:00,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277
5,231808869,2023-04-04,2020-12-01,23:00:00,18,Southeast,1826,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,9900 COMPTON AV,NaN,33.9467,-118.2463
6,230110144,2023-04-04,2020-07-03,09:00:00,1,Central,182,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1100 S GRAND AV,NaN,34.0415,-118.2620
7,220314085,2022-07-22,2020-05-12,11:10:00,3,Southwest,303,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,2500 S SYCAMORE AV,NaN,34.0335,-118.3537
8,231309864,2023-04-28,2020-12-09,14:00:00,13,Newton,1375,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1300 E 57TH ST,NaN,33.9911,-118.2521
9,211904005,2020-12-31,2020-12-31,12:20:00,19,Mission,1974,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,9000 CEDROS AV,NaN,34.2336,-118.4535


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986500 entries, 0 to 986499
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id_report         986500 non-null  int64         
 1   date_report       986500 non-null  datetime64[ns]
 2   date_ocurrance    986500 non-null  datetime64[ns]
 3   time_ocurrance    986500 non-null  object        
 4   area_code         986500 non-null  int64         
 5   area_name         986500 non-null  object        
 6   district          986500 non-null  int64         
 7   crime_severity    986500 non-null  int64         
 8   crime_code        986500 non-null  int64         
 9   crime_desc        986500 non-null  object        
 10  crime_mo          840065 non-null  object        
 11  age_victim        986500 non-null  int64         
 12  sex_victim        846925 non-null  object        
 13  descent_victim    846914 non-null  object        
 14  loc_

### area_code

### area_name

### age_victim

Mostrar la información de los no nulos de la columna age_victim

In [28]:
df['age_victim'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 986500 entries, 0 to 986499
Series name: age_victim
Non-Null Count   Dtype
--------------   -----
986500 non-null  int64
dtypes: int64(1)
memory usage: 7.5 MB


Mostrar la información de los valores únicos de la columna

In [29]:
if 'age_victim' in df.columns:
    print(sorted(df['age_victim'].dropna().unique()))
    print("\n")
else:
    print("Column 'age_victim' not found in the dataframe.\n")

[np.int64(-4), np.int64(-3), np.int64(-2), np.int64(-1), np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66), np.int64(67), np.int64(68), 

Mostrar la información de los valores negativos

In [30]:
df['age_victim'].loc[df['age_victim']<0]

143      -2
375      -2
581      -1
595      -1
618      -1
         ..
636182   -1
654233   -1
681376   -1
728608   -1
828481   -1
Name: age_victim, Length: 132, dtype: int64

Mostrar el porcentaje de valores negativos en la

In [31]:
(df['age_victim'] < 0).sum() / df['age_victim'].notna().sum() * 100

np.float64(0.01338063862138875)

### location

In [32]:
df['location'].unique()

array(['1900 S  LONGWOOD                     AV',
       '1000 S  FLOWER                       ST',
       '1400 W  37TH                         ST', ...,
       '9500    LA CIENEGA                   BL',
       '1800 E  MARTIN LUTHER KING JR        BL',
       '12200    ART                          ST'],
      shape=(66322,), dtype=object)

In [33]:
df['location'] = df['location'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))    #la funcion re.sub es para reemplazar (lo que quieres reemplazar, por lo que lo vas a reemplazar)

In [34]:
df['location'] = df['location'].apply(str.strip).apply(str.lower)    # Para quitar espacios y ponerlo en minuscula

In [35]:
df.head()

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,case_status_code,case_status_desc,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
0,190326475,2020-03-01,2020-03-01,21:30:00,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 s longwood av,NaN,34.0375,-118.3506
1,200106753,2020-02-09,2020-02-08,18:00:00,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 s flower st,NaN,34.0444,-118.2628
2,200320258,2020-11-11,2020-11-04,17:00:00,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 w 37th st,NaN,34.0210,-118.3002
3,200907217,2023-05-10,2020-03-10,20:37:00,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 riverside dr,NaN,34.1576,-118.4387
4,220614831,2022-08-18,2020-08-17,12:00:00,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 transient,NaN,34.0944,-118.3277


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986500 entries, 0 to 986499
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id_report         986500 non-null  int64         
 1   date_report       986500 non-null  datetime64[ns]
 2   date_ocurrance    986500 non-null  datetime64[ns]
 3   time_ocurrance    986500 non-null  object        
 4   area_code         986500 non-null  int64         
 5   area_name         986500 non-null  object        
 6   district          986500 non-null  int64         
 7   crime_severity    986500 non-null  int64         
 8   crime_code        986500 non-null  int64         
 9   crime_desc        986500 non-null  object        
 10  crime_mo          840065 non-null  object        
 11  age_victim        986500 non-null  int64         
 12  sex_victim        846925 non-null  object        
 13  descent_victim    846914 non-null  object        
 14  loc_

### cross_street

In [37]:
df['cross_street'].unique()

array([nan, 'VANOWEN                      AV', 'INGLEWOOD', ...,
       'CHARLES E YOUNG              DR',
       'LUCERNE                      ST', '222ND'],
      shape=(10338,), dtype=object)

In [38]:
df['cross_street'] = df['cross_street'].astype(str)

In [39]:
df['cross_street'] = df['cross_street'].apply(str.strip).apply(str.lower)    # Para quitar espacios y ponerlo en minuscula

In [40]:
df.tail()

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,case_status_code,case_status_desc,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
986495,242011172,2024-08-20,2024-08-17,23:00:00,20,Olympic,2033,1,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",...,IC,Invest Cont,341.0,NaN,NaN,NaN,3700 wilshire bl,nan,34.0617,-118.3066
986496,240710284,2024-07-24,2024-07-23,14:00:00,7,Wilshire,788,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,4000 w 23rd st,nan,34.0362,-118.3284
986497,240104953,2024-01-15,2024-01-15,01:00:00,1,Central,101,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,NaN,NaN,NaN,1300 w sunset bl,nan,34.0685,-118.2460
986498,240309674,2024-04-24,2024-04-24,15:00:00,3,Southwest,358,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230.0,NaN,NaN,NaN,flower st,jefferson bl,34.0215,-118.2868
986499,240910892,2024-08-13,2024-08-12,23:00:00,9,Van Nuys,914,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,6900 vesper av,nan,34.1961,-118.4510


### latitude

### longitude

## Investigación - Case Status Code/Description

In [ ]:
df = pd.read_csv('Crime_Data_from_2020_to_Present.csv') 
unique_status = df['Status'].unique()
print("Valores únicos en la columna 'Status':")
print(unique_status)

non_standard_status = df[df['Status'].isin(['AO', 'JO'])]
result = non_standard_status[['Status', 'Status Desc']].head(5)
print(result)

In [ ]:
status_not_null = df['Status'].notna()
total_status = status_not_null.sum()

non_standard_count = df['Status'].isin(['AO', 'JO']).sum()
percentage = (non_standard_count / total_status) * 100

print(f"Cantidad total de valores no estándar ('AO' y 'JO'): {non_standard_count}")
print(f"Porcentaje respecto al total de valores definidos en 'Status': {percentage:.2f}%")

In [ ]:
null_count = df['Status'].isna().sum()
total_rows = len(df)
null_percentage = (null_count / total_rows) * 100

print(f"Cantidad de valores nulos en 'Status': {null_count}")
print(f"Porcentaje respecto al total de filas: {null_percentage:.2f}%")

In [ ]:
null_count_desc = df['Status Desc'].isna().sum()
total_rows = len(df)
null_percentage_desc = (null_count_desc / total_rows) * 100

print(f"Cantidad de valores nulos en 'Status Desc': {null_count_desc}")
print(f"Porcentaje respecto al total de filas: {null_percentage_desc:.2f}%")

In [ ]:

status_null_rows = df[df['Status'].isna()]
print(status_null_rows[['Status Desc']])